# Creating Interactive Plots for Heatmaps of Avg. Probability vs Class and Time

### Objective: To display the average probability vs. class and time as a heat map that uses the interactive tools provided by Bokeh with the hopes of making the data more visually appealing and easily accessible.

# Imports

In [27]:
%matplotlib inline
import sys
import os
import requests
import datetime
import dateutil.parser
import logging
import json
import numpy
import pandas
import matplotlib
from matplotlib import pyplot
import seaborn
import sklearn
import sklearn.metrics
import numpy as np
from bokeh.models import ColumnDataSource, CustomJS, Range1d, Select, Dropdown, LabelSet, ColorBar, LinearColorMapper
from bokeh.plotting import figure, output_notebook, show, output_file, save
from bokeh.palettes import Sunset11, Magma256
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [28]:
path_to_elasticc_metrics = '/astro/users/joseph02/elasticc_metrics'

In [29]:
sys.path.append(path_to_elasticc_metrics)

In [30]:
from metric_querier import ELAsTiCCMetricsQuerier 

Note that for this last import, ensure that the path to your respective folder in which you're running this notebook is correct.

# Account Logistics

To perform needed queries, talk to rob knop (raknop@lbl.gov) about setting up an account.

In [31]:
logger = logging.getLogger(__name__)
if not logger.hasHandlers():
    _logout = logging.StreamHandler( sys.stderr )
    logger.addHandler( _logout )
    _formatter = logging.Formatter( f'[%(asctime)s - %(levelname)s] - %(message)s', datefmt='%Y-%m-%d %H:%M:%S' )
    _logout.setFormatter( _formatter )

logger.setLevel( logging.DEBUG )

In [32]:
username = "josephy"
with open( os.path.join(os.getenv("HOME"), "josephy_passwd.txt")) as ifp:
    password = ifp.readline().strip()
    
emq = ELAsTiCCMetricsQuerier( tomusername=username, tompasswd=password, logger=logger )

# Loading in data

In [33]:
probhist = emq.probhist()

[2023-06-06 14:15:25 - DEBUG] - Sending query to get probabilistic metrics histogram table
[2023-06-06 14:15:40 - DEBUG] - Got response, pandifying
[2023-06-06 14:15:44 - DEBUG] - Done


We recommend saving `probhist` as a csv file for easy access later on (e.g. `probhist.to_csv('probhist.csv')`).

Load in the histogram table. Run `help(ELAsTiCCMetricsQuerier)` to get more details on the 
contents of the table above.

# Define our Classifier and True Classes

In [36]:
cferstodo = [40, 89, 44]
trueclasstodo = [ 111, 112, 113, 131 ]
print( f"Going to do classifiers: {cferstodo}" )
print( f"Going to do classes: {[emq.classname[i] for i in trueclasstodo]}" )

Going to do classifiers: [40, 89, 44]
Going to do classes: ['Ia', 'Ib/c', 'II', 'SLSN']


In this notebook we only use one classifier at a time. 

In [37]:
trueClassResults = {}
for cfer in cferstodo[:1]:
    brokername = emq.classifier_info[cfer]['brokerName']
    brokerversion = emq.classifier_info[cfer]['brokerVersion']
    classifiername = emq.classifier_info[cfer]['classifierName']
    classifierparams = emq.classifier_info[cfer]['classifierParams']
    cfertitle = f'{brokername} {brokerversion} {classifiername} {classifierparams}'
    for trueclass in trueclasstodo:
        logger.debug( f"doing cfer {cfer}, true class {emq.classname[trueclass]}" )
        title = f"True Class {emq.classname[trueclass]} for {cfertitle}"
        subdf = probhist.xs( ( cfer, trueclass ), level=( 'classifierId', 'trueClassId' ) )
        probdf = subdf / subdf.groupby( ['classId', 'tbin'] ).sum()
        probdf.reset_index( inplace=True )
        probdf.set_index( ['tbin', 'probbin', 'classId'], inplace=True )
        # logger.debug( f"Before filling in missing indexes, len(probhist) = {len(probhist)}" )
        # Fill in missing indexes with 0s.  This is a bit slow (takes a couple of seconds each time).
        # new_indices = pandas.MultiIndex.from_product( [ range( 0, emq.tbin_num+2 ), range( 0, emq.probbin_num+2 ), 
        #                                                 probdf.index.get_level_values('classId') ] )
        # probdf.reindex( new_indices, fill_value=0. )
        # logger.debug( f"After filling in missing indexes, len(probhist) = {len(probhist)}" )
        # Fill in the probability values associated with the bins
        probdf['prob'] = emq.probbin_val( probdf.reset_index()['probbin'] )
        # HACK ALERT.  There's this issue with the histogramming.  Probably (but not certainly) lots
        # of the things that show up in probbin=1 are really 0 probabilities reported by the brokers,
        # but now that we've binned it we're going to treat it as 0.025.  Big deal, you say, until
        # you realize that there might be 15 classes that all were given 0 probability by a
        # broker, and now you've got an extra 15*0.025=0.625 of probability that's not real!  So, 
        # we're going to make the rash assumption that the probability value associated with 
        # probbin=1 is 0, not 2.5%, which is wrong, but hopefully less wrong than a direct
        # interpretation of the histogram.
        probdf.loc[ pandas.IndexSlice[ :, 1, : ] ] = 0.
        # Don't need probbin any more
        probdf.reset_index( inplace=True )
        probdf.drop( [ 'probbin' ] , axis=1, inplace=True )
        probdf.set_index( [ 'tbin', 'classId' ], inplace=True )
        
        # Finally, combine things together to get a total probability
        #   as a function of classId and tbin
        probdf['prob'] *= probdf['count']
        probdf.drop( [ 'count' ], axis=1, inplace=True )
        probdf = probdf.groupby( [ 'tbin', 'classId' ] ).apply( sum )

        # To check how much probability we've lost with our assuming probbin1 is 0 probability,
        # do probdf.groupby(['tbin']).sum()

        # Fill in the missing tbins
        # logger.debug( f"Before fliling in missing tbins, len(probdf) = {len(probdf)}" )
        new_indices = pandas.MultiIndex.from_product( [ range(0, emq.tbin_num+2), probdf.index.get_level_values( 'classId' ) ] )
        probdf.reindex( new_indices, fill_value=0. )
        # logger.debug( f"After filling in missing tbins, len(probdf) = {len(probdf)}" )
        trueClassResults[emq.classname[trueclass]] = probdf

[2023-06-06 14:15:44 - DEBUG] - doing cfer 40, true class Ia
[2023-06-06 14:15:45 - DEBUG] - doing cfer 40, true class Ib/c
[2023-06-06 14:15:45 - DEBUG] - doing cfer 40, true class II
[2023-06-06 14:15:45 - DEBUG] - doing cfer 40, true class SLSN


## Initial Attempt at Plotting trueClassResults 

The two cells below manually organizes the data for the plot. We take the binned data and turn into a 2d matrix of time versus class which we plot as a heat map.

In [38]:
probdf = trueClassResults['SLSN']
prob2df = trueClassResults['II']
prob3df = trueClassResults['Ib/c']
prob4df = trueClassResults['Ia']

In [39]:
print(probdf)

                  prob
tbin classId          
0    110      0.003388
     111      0.006016
     112      0.019934
     113      0.149386
     114      0.032940
...                ...
27   213      0.000001
     214      0.000006
     215      0.000000
     220      0.000000
     221      0.005942

[700 rows x 1 columns]


In [40]:
grid_tab = pandas.pivot_table(probdf,values="prob", columns="tbin", index="classId")
grid_tab2 = pandas.pivot_table(prob2df,values="prob", columns="tbin", index="classId")
grid_tab3 = pandas.pivot_table(prob3df,values="prob", columns="tbin", index="classId" )
grid_tab4 = pandas.pivot_table(prob4df,values="prob", columns="tbin", index="classId" )

The following print statement is merely to provide a better understanding as to what `grid_tab` is.

In [41]:
print(grid_tab)

tbin           0         1         2         3         4         5         6    
classId                                                                         
110      0.003388  0.002661  0.002339  0.002052  0.001510  0.001442  0.001128  \
111      0.006016  0.005841  0.004029  0.004079  0.003098  0.002651  0.002253   
112      0.019934  0.018960  0.016356  0.015628  0.014062  0.013024  0.011021   
113      0.149386  0.153522  0.153344  0.145795  0.146062  0.134648  0.127118   
114      0.032940  0.034051  0.033591  0.029859  0.028645  0.024022  0.019391   
115      0.000158  0.000188  0.000372  0.000327  0.000210  0.000076  0.000067   
120      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
121      0.000028  0.000036  0.000004  0.000085  0.000028  0.000000  0.000000   
122      0.000019  0.000012  0.000007  0.000006  0.000003  0.000027  0.000000   
123      0.000295  0.000250  0.000070  0.000177  0.000176  0.000153  0.000294   
124      0.008435  0.005162 

Now we plot. In order for us to better see the variation in probability, we thought it would be appropriate to stick with the Bokeh palette `Magma256` but run `.reverse()` on a list version of the palette. The results are shown below.

In [42]:
new_magma = list(Magma256)
new_magma.reverse()
# Here we create an inverse of the original palette to more easily show differences in data
cfertitle = f'{brokername} {brokerversion} {classifiername} {classifierparams}'
color_mapper = LinearColorMapper(palette=new_magma, low=0, high=1)
p = figure(width=670, height=400, toolbar_location=None,
           title=f"True Class: SLSN; {cfertitle}", x_axis_label = "Δt rel. peak (days)", y_axis_label = "Class")
plot_1 = p.image(image=[grid_tab.values[:,:]], x=0, y=0, dw=28, dh=25,color_mapper=color_mapper, level="image")
plot_2 = p.image(image=[grid_tab2.values[:,:]], x=0, y=0, dw=28, dh=25,color_mapper=color_mapper, level="image")
plot_3 = p.image(image=[grid_tab3.values[:,:]], x=0, y=0, dw=28, dh=25,color_mapper=color_mapper, level="image")
plot_4 = p.image(image=[grid_tab4.values[:,:]], x=0, y=0, dw=28, dh=25,color_mapper=color_mapper, level="image")

plot_2.visible = False
plot_3.visible = False
plot_4.visible = False
# These plot_#.visible commands allow us to only view one plot at a time

plot_dict = {}
plot_dict["SLSN"] = plot_1
plot_dict["II"] = plot_2
plot_dict["Ib/c"] = plot_3
plot_dict["Ia"] = plot_4

# dropdown widget + Javascript code for interactivity
select = Select(title="Plot to show:", value="Line 1", options=["SLSN", "II", "Ib/c", "Ia"])
select.js_on_change("value", CustomJS(args=dict(plot_dict=plot_dict, p=p, title_text=cfertitle), code="""
    const titles = {
        "SLSN": "True Class: SLSN; " + title_text,
        "II": "True Class: II; " + title_text,
        "Ib/c": "True Class: Ib/c; " + title_text,
        "Ia": "True Class: Ia; " + title_text
    };

    for (const plot of Object.values(plot_dict)) {
        plot.visible = false;
    }

    plot_dict[this.value].visible = true;
    p.title.text = titles[this.value];
"""))

# The javascript above ensures that the title of the selected graph matches that associated data.

layout = column(select, p)
color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), title = 'Probability', title_text_align = 'center')
p.add_layout(color_bar, 'right')
y_tick = np.arange(25)
p.yaxis.ticker = y_tick
name_arr = [emq.classname[i] for i in grid_tab.index.values]
p.yaxis.major_label_overrides = {i:name for i, name in zip(y_tick, name_arr)}
x_tick = np.arange(0, 28, 4) + 0.5
p.xaxis.ticker = x_tick
time_arr = emq.tbin_val(x_tick)
p.xaxis.major_label_overrides = {i:str(name) for i, name in zip(x_tick, time_arr)}

show(layout)
#output_file(filename='/epyc/users/joseph02/public_html/elasticc_interactive_heatmap.html')
#save(layout)

Here's a modified version of the plot above, easily extended to as many classes as you want. 

In [43]:
cferstodo = [40, 89, 44]
trueclasstodo = [111, 112, 113, 131, 213, 214, 221]
print( f"Going to do classifiers: {cferstodo}" )
print( f"Going to do classes: {[emq.classname[i] for i in trueclasstodo]}" )

Going to do classifiers: [40, 89, 44]
Going to do classes: ['Ia', 'Ib/c', 'II', 'SLSN', 'Delta Scuti', 'EB', 'AGN']


In [44]:
trueClassResults = {}
for cfer in cferstodo[:1]:
    brokername = emq.classifier_info[cfer]['brokerName']
    brokerversion = emq.classifier_info[cfer]['brokerVersion']
    classifiername = emq.classifier_info[cfer]['classifierName']
    classifierparams = emq.classifier_info[cfer]['classifierParams']
    cfertitle = f'{brokername} {brokerversion} {classifiername} {classifierparams}'
    for trueclass in trueclasstodo:
        logger.debug( f"doing cfer {cfer}, true class {emq.classname[trueclass]}" )
        title = f"True Class {emq.classname[trueclass]} for {cfertitle}"
        subdf = probhist.xs( ( cfer, trueclass ), level=( 'classifierId', 'trueClassId' ) )
        probdf = subdf / subdf.groupby( ['classId', 'tbin'] ).sum()
        probdf.reset_index( inplace=True )
        probdf.set_index( ['tbin', 'probbin', 'classId'], inplace=True )
        # logger.debug( f"Before filling in missing indexes, len(probhist) = {len(probhist)}" )
        # Fill in missing indexes with 0s.  This is a bit slow (takes a couple of seconds each time).
        # new_indices = pandas.MultiIndex.from_product( [ range( 0, emq.tbin_num+2 ), range( 0, emq.probbin_num+2 ), 
        #                                                 probdf.index.get_level_values('classId') ] )
        # probdf.reindex( new_indices, fill_value=0. )
        # logger.debug( f"After filling in missing indexes, len(probhist) = {len(probhist)}" )
        # Fill in the probability values associated with the bins
        probdf['prob'] = emq.probbin_val( probdf.reset_index()['probbin'] )
        # HACK ALERT.  There's this issue with the histogramming.  Probably (but not certainly) lots
        # of the things that show up in probbin=1 are really 0 probabilities reported by the brokers,
        # but now that we've binned it we're going to treat it as 0.025.  Big deal, you say, until
        # you realize that there might be 15 classes that all were given 0 probability by a
        # broker, and now you've got an extra 15*0.025=0.625 of probability that's not real!  So, 
        # we're going to make the rash assumption that the probability value associated with 
        # probbin=1 is 0, not 2.5%, which is wrong, but hopefully less wrong than a direct
        # interpretation of the histogram.
        probdf.loc[ pandas.IndexSlice[ :, 1, : ] ] = 0.
        # Don't need probbin any more
        probdf.reset_index( inplace=True )
        probdf.drop( [ 'probbin' ] , axis=1, inplace=True )
        probdf.set_index( [ 'tbin', 'classId' ], inplace=True )
        
        # Finally, combine things together to get a total probability
        #   as a function of classId and tbin
        probdf['prob'] *= probdf['count']
        probdf.drop( [ 'count' ], axis=1, inplace=True )
        probdf = probdf.groupby( [ 'tbin', 'classId' ] ).apply( sum )

        # To check how much probability we've lost with our assuming probbin1 is 0 probability,
        # do probdf.groupby(['tbin']).sum()

        # Fill in the missing tbins
        # logger.debug( f"Before fliling in missing tbins, len(probdf) = {len(probdf)}" )
        new_indices = pandas.MultiIndex.from_product( [ range(0, emq.tbin_num+2), probdf.index.get_level_values( 'classId' ) ] )
        probdf.reindex( new_indices, fill_value=0. )
        # logger.debug( f"After filling in missing tbins, len(probdf) = {len(probdf)}" )
        trueClassResults[emq.classname[trueclass]] = probdf

[2023-06-06 14:15:46 - DEBUG] - doing cfer 40, true class Ia
[2023-06-06 14:15:46 - DEBUG] - doing cfer 40, true class Ib/c
[2023-06-06 14:15:47 - DEBUG] - doing cfer 40, true class II
[2023-06-06 14:15:47 - DEBUG] - doing cfer 40, true class SLSN
[2023-06-06 14:15:47 - DEBUG] - doing cfer 40, true class Delta Scuti
[2023-06-06 14:15:47 - DEBUG] - doing cfer 40, true class EB
[2023-06-06 14:15:48 - DEBUG] - doing cfer 40, true class AGN


In [45]:
new_magma = list(Magma256)
new_magma.reverse()
color_mapper = LinearColorMapper(palette=new_magma, low=0, high=1)
# Here we create an inverse of the original palette to more easily show differences in data
cfertitle = f'{brokername} {brokerversion} {classifiername} {classifierparams}'

pivot_tables = {}
for key, value in trueClassResults.items():
    pivot_table = pandas.pivot_table(value, values="prob", columns="tbin", index="classId")
    pivot_tables[key] = pivot_table


p = figure(width=670, height=400, toolbar_location=None, title=f"True Class: {list(pivot_tables.keys())[0]}; {cfertitle}",
           x_axis_label="Δt rel. peak (days)", y_axis_label="Class")

plot_dict = {}
for key, pivot_table in pivot_tables.items():
    plot = p.image(image=[pivot_table.values[:, :]], x=0, y=0, dw=28, dh=25, color_mapper=color_mapper, level="image")
    plot.visible = False
    plot_dict[key] = plot

list(plot_dict.values())[0].visible = True

select = Select(title="Plot to show:", value="Line 1", options=list(plot_dict.keys()))
select.js_on_change("value", CustomJS(args=dict(plot_dict=plot_dict, p=p, title_text=cfertitle), code="""

    for (const plot of Object.values(plot_dict)) {
        plot.visible = false;
    }

    plot_dict[this.value].visible = true;
    p.title.text = "True Class: " + this.value + "; " + title_text;
"""))


layout = column(select, p)
color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0), title='Probability', title_text_align='center')
p.add_layout(color_bar, 'right')
y_tick = np.arange(25)
p.yaxis.ticker = y_tick
name_arr = [emq.classname[i] for i in pivot_table.index.values]
p.yaxis.major_label_overrides = {i: name for i, name in zip(y_tick, name_arr)}
x_tick = np.arange(0, 28, 4) + 0.5
p.xaxis.ticker = x_tick
time_arr = emq.tbin_val(x_tick)
p.xaxis.major_label_overrides = {i: str(name) for i, name in zip(x_tick, time_arr)}


show(layout)
#output_file(filename='/epyc/users/joseph02/public_html/bokeh_example_for_elasticc.html',
            #title='Interactive Probability Histograms')
#save(layout)


# Next Steps

While this was an effective way of reducing the numeber of plots shown at once (thus reducing visual clutter) it can still be imporved upon.

Sticking with the dropdown widget, we could start by adding multiple classifiers to compare and contrast their performance at identifying the same true class. This can be done by adding a multiselect tool that allows the user to pick precisely which classifiers and true classes they wish to analyze. To look solely at the graphs above, visit this link: https://epyc.astro.washington.edu/~joseph02/